In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
#guides from home page and github
#https://catboost.ai/
#https://github.com/catboost/catboost

from catboost import Pool, CatBoostClassifier

/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
#use the features after feature selection
x= np.load('feature_train1.npy')
y= np.load('labels.npy')

In [4]:
# make scaler to reshape the size of x 
scaler = StandardScaler()
x =scaler.fit_transform(x)

In [5]:
#reshape and tune the data

pv_idx =np.where(y[:,-1]!=1.)[0]
nv_idx =np.where(y[:,-1] ==1.)[0]

#use positive and negative 
pv=x[pv_idx]
nv=x[nv_idx]
rp_num =int((nv.shape[0] -pv.shape[0])/pv.shape[0])
rp_pv = np.repeat(pv,2,axis=0)
x=np.concatenate((x,rp_pv))

y_pv=y[pv_idx]
y_nv=y[nv_idx]
y_rp_pv = np.repeat(y_pv,2,axis=0)
y=np.concatenate((y,y_rp_pv))
y.shape


(40713, 101)

In [6]:
#Use the largest and possible number from 101 dimensions
class_labels = np.argmax(y, axis=1)
class_labels[:10]

array([ 36,  45, 100,  97,   2, 100, 100, 100, 100,   2])

In [7]:
x_train,x_test, y_train, y_test = train_test_split(x, class_labels, test_size=0.3, random_state=1)

In [8]:
#the use of the pool function from CatBoost website
#https://catboost.ai/en/docs/concepts/python-reference_pool
train_pool = Pool(data=x_train, label=y_train)
test_pool = Pool(data=x_test, label=y_test) 

In [11]:
#https://datascience.stackexchange.com/questions/67548/getting-feature-vectors-from-catboost-pool
#use classifier after pool
model = CatBoostClassifier(
    #iteration test
    iterations=2000,
    learning_rate=0.1,
    random_strength=0.1,
    depth=10,
    loss_function='MultiClass',
    eval_metric='Accuracy',
    leaf_estimation_method='Newton',
    task_type="GPU"
)

In [17]:
model.fit(train_pool,plot=True,eval_set=test_pool)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.4809642	test: 0.4811692	best: 0.4811692 (0)	total: 681ms	remaining: 22m 41s
1:	learn: 0.4871048	test: 0.4889471	best: 0.4889471 (1)	total: 1.33s	remaining: 22m 5s
2:	learn: 0.4918418	test: 0.4941051	best: 0.4941051 (2)	total: 1.99s	remaining: 22m 7s
3:	learn: 0.4965437	test: 0.4983625	best: 0.4983625 (3)	total: 2.65s	remaining: 22m 2s
4:	learn: 0.4982982	test: 0.5004094	best: 0.5004094 (4)	total: 3.29s	remaining: 21m 54s
5:	learn: 0.5045089	test: 0.5055674	best: 0.5055674 (5)	total: 3.94s	remaining: 21m 50s
6:	learn: 0.5086845	test: 0.5090879	best: 0.5090879 (6)	total: 4.6s	remaining: 21m 48s
7:	learn: 0.5118776	test: 0.5117079	best: 0.5117079 (7)	total: 5.26s	remaining: 21m 48s
8:	learn: 0.5154216	test: 0.5154740	best: 0.5154740 (8)	total: 5.92s	remaining: 21m 50s
9:	learn: 0.5185796	test: 0.5184215	best: 0.5184215 (9)	total: 6.58s	remaining: 21m 49s
10:	learn: 0.5227552	test: 0.5221877	best: 0.5221877 (10)	total: 7.24s	remaining: 21m 48s
11:	learn: 0.5270711	test: 0.52660

In [15]:
test_pred=model.predict(x_test)
#reshape to make y_pred in the same shape
y_pred = test_pred.reshape(-1)
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html
f1_score(y_true=y_test,y_pred=y_pred, average='weighted')

0.9307865071190632

In [19]:
pred =model.predict(x_train)
f1_score(y_true=y_train,y_pred=pred,average='weighted')

0.9809752152919542

In [22]:
test_data = np.load('feature_test1.npy')
test_data = scaler.transform(test_data)
predd =model.predict(test_data)
#put the result in list form and directly and create a csvs object
predd=predd.tolist()


In [23]:
df= pd.DataFrame(predd)
df

,0
0,4
1,26
2,31
3,38
4,32
...,...
795,64
796,83
797,67
798,32


In [24]:
df.to_csv('predict.csv')